In [ ]:
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import tensorflow as tf

from Thesis_PreProcessing import trials_note as tn
from Thesis_PreProcessing import ico_codes as ic
from Thesis_PreProcessing import more_io as mi

import pickle


In [ ]:

def RNN_Dense_Representation_graph(state_size = 200, input_features = 1243, 
                                    batch_size = 30, target_features = 943,
                                    max_steps = 40, learning_rate = 1e-3, test=False):
    
    reset_g()
    
    inputs = tf.placeholder(tf.float32, [None, max_steps, input_features], name="inputs")
    targets = tf.placeholder(tf.float32, [None, max_steps, target_features], name="targets")
    
    with tf.name_scope("Projection_Layer_1") as scope:                    
        fc2_inputs = tf.contrib.layers.fully_connected(inputs, 1000)
    with tf.name_scope("Projection_Layer_2") as scope:
        lstm_inputs = tf.contrib.layers.fully_connected(fc2_inputs, state_size)
    lstm_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
  
    with tf.name_scope("LSTM"):                                                           
        init_state= lstm_cell.zero_state(batch_size, tf.float32)        
        lstm_outputs, fin_states = tf.nn.dynamic_rnn(cell=lstm_cell,initial_state=init_state,  
                                                    dtype=tf.float32, sequence_length=length(inputs), inputs=lstm_inputs)
                    
    
    with tf.variable_scope('sigmoid'):
        W = tf.get_variable('W', [state_size, target_features])
        b = tf.get_variable('b', [target_features], initializer=tf.constant_initializer(0.0))
 
    with tf.name_scope("Sigmoid"):
        lstm_outputs = tf.reshape(lstm_outputs, [-1, state_size]) 
   
        new_y = tf.reshape(targets, [-1, target_features])
        scores = tf.matmul(lstm_outputs, W) + b
        predictions = tf.nn.sigmoid(scores)
    
        total_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=scores, labels=new_y))
                            
    if not test:
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    else:
        train_step = []
    
 
    return dict(
        inputs = inputs,
        targets = targets,
        total_loss = total_loss,
        train_step = train_step,

        initial = init_state,
        final = fin_states,
        predictions = predictions,
        saver = tf.train.Saver()
    )
    

In [ ]:
def length(inputs):
    used = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length


def dict_to_paddedY(ico_dict, keys, max_length = 40, feat_nums=943):
    
    dim = len(keys)
    
    X = np.zeros((dim, max_length, feat_nums))
    Y = np.zeros((dim, max_length, feat_nums))
    
    i = 0
    for key in ico_dict:
        if key in keys:
            j = 0
            dict_2 = ico_dict[key]
            keys2 = list(dict_2.keys())
            for key2 in dict_2:
                X[i, j, :] = dict_2[key2]
                if j == (len(keys2) - 1):
                    Y[i, j, -1] = 1
                else:
                    Y[i, j, :] = dict_2[keys2[j+1]]
                j += 1   
            i += 1
        else:
            continue 
            
    return Y

def dense_representation_add(ico_dict, note_dict, subjects, max_length=40, feat_num=1243):
    dim = len(subjects)
    X = np.zeros((dim, max_length, feat_num))
    i = 0
    for subject in ico_dict:
        if subject in subjects:
            if subject in note_dict:
                j = 0
                visit_dict_ico = ico_dict[subject]
                for visit in visit_dict_ico:
                    ico_vec = ico_dict[subject][visit]
                    note_vec = note_dict[subject][visit][0]
                    input_vec = np.concatenate((ico_vec, note_vec))
                    X[i, j, :] = input_vec
                    j += 1  
                i += 1
            else:
                print("WRONG!")
    return X
     
    

In [ ]:
# Training/Session
def reset_g():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
def train_baseline(g, inputs, targets, epoch_nums=2, batch_size=30, verbose=True, save=False):
    tf.set_random_seed(1234)
    train_size = inputs.shape[0]
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_losses = []
        
        for i in range(epoch_nums):
            epoch_training_loss = 0
            idx = np.random.choice(train_size, batch_size, replace=False)
            training_loss = 0
            training_state = None
            num_iters = int(train_size/batch_size)
            for j in range(num_iters):
                
                X = inputs[j*batch_size:(j+1)*batch_size]
                Y = targets[j*batch_size:(j+1)*batch_size]
                feed_dict={g['inputs']: X, g['targets']: Y}
                
                if training_state is not None:
                   
            
                    training = training_state
                    feed_dict[g['initial']] = training
                    
                    
                cur_training_loss, training_state, _ = sess.run([g['total_loss'],
                                                      g['final'],
                                                      g['train_step']],
                                                             feed_dict)
                training_loss += cur_training_loss
                
            if verbose:
                print("Average training loss for Epoch", idx, ":", training_loss/num_iters)
            training_losses.append(training_loss/num_iters)

        if isinstance(save, str):
            g['saver'].save(sess, save)


In [ ]:
# Inference/Testing
def test_baseline(g, checkpoint, X_test, Y_test, N=20):
    
    with tf.Session() as sess:
        
        sess.run(tf.initialize_all_variables())
        g['saver'].restore(sess, checkpoint)
        
        num_iters = X_test.shape[0]
        state = None
        total_test_loss = 0
        recall_accs_20 = []
        recall_accs_25 = []
        recall_accs_30 = []
        avg_loss = []
        for i in range(num_iters):
            X = X_test[i:i+1,:,:]
            targets = Y_test[i:i+1, :, :]
            if state is not None:
                state_fw, state_bw = state
                feed_dict={g['inputs']: X, g['targets']: targets, g['initial']: state}
            else:
                feed_dict={g['inputs']: X, g['targets']: targets}
            cur_test_loss, state, predictions, _ = sess.run([g['total_loss'],
                                                      g['final'],
                                                      g['predictions'],
                                                      g['train_step']],
                                                             feed_dict)
            recall_acc_20 = recall_at_n(predictions, targets, n=20)
            recall_acc_25 = recall_at_n(predictions, targets, n=25)
            recall_acc_30 = recall_at_n(predictions, targets, n=30)
            recall_accs_20.append(recall_acc_20)
            recall_accs_25.append(recall_acc_25)
            recall_accs_30.append(recall_acc_30)
            total_test_loss += cur_test_loss
            avg_loss.append(total_test_loss/float(i+1))
            
        avg_test_loss = total_test_loss/num_iters
        
        recall_20 = np.nanmean(recall_accs_20)
        recall_25 = np.nanmean(recall_accs_25)
        recall_30 = np.nanmean(recall_accs_30)
        
        writer = tf.summary.FileWriter("tmp/model_dense_7")
        writer.flush()
        writer.add_graph(sess.graph)
        writer.close()
        
        return [avg_loss, avg_test_loss, recall_20, recall_25, recall_30]
     


    
def recall_at_n(predictions, targets, n=20):
    dim = targets.shape[1]

    pred_dim = 0
    eps = .000000001
    for i in range(dim):
        #print "hello"
        if targets[0, i, -1] == 1:
      
            pred_dim = i
          
            break
    if pred_dim != 0:
        valid_targets = targets[0, 0:pred_dim, :]
        valid_preds = predictions[0:pred_dim, :]
        recall_val = np.zeros((pred_dim,))
        for i in range(pred_dim):
     
            pos_count = 0.0
            pred_pos = 0.0
            
            for j in range(n):
                if np.sort(-valid_targets)[i][j] <= (-1. + eps) and np.sort(-valid_targets)[i][j] >= (-1. - eps):
                    pos_count += 1.
                  
            for j in range(n):
                k = int(pos_count)
                if np.argsort(-valid_preds)[i][j] in np.argsort(-valid_targets)[i][0:k]:
                    pred_pos += 1.
                    
            if pos_count != 0.0:
                recall_val[i] = pred_pos/float(pos_count)
            else:
                recall_val[i] = 0.0
           
        return np.mean(recall_val)
    else:
        return np.nan

    

In [ ]:
def cross_validation(X_train, Y_train, X_val, Y_val, s_size = [200, 300, 400], b_size = [20, 30, 40], learn_rate = 1e-3):
    for ss in s_size:
        for bs in b_size:
            print("State size: " + str(ss) + " Batch_Size: " + str(bs))
            g = RNN_Dense_Representation_graph(state_size=ss, batch_size = bs, learning_rate=learn_rate)
            train_baseline(g, X_train, Y_train, batch_size=bs, save="/tmp/dense_val.ckpt")
            g_2 = RNN_Dense_Representation_graph(state_size=ss, learning_rate=learn_rate, batch_size=1, test=True)
            losses, loss, recall_20, recall_25, recall_30 = test_baseline(g_2, "/tmp/dense_val.ckpt", X_val, Y_val)
            print(loss)
            print(recall_20, recall_25, recall_30)
            
 

In [ ]:
# ICD Data
current_connection = tn.connect()
current_cur = current_connection[1]

ico_dict = ic.ico_dict_populator(current_cur)
ico_dict_keys = list(ico_dict.keys())
ico_dict_keys = np.array(ico_dict_keys)
tr_idx = int(.7*len(ico_dict_keys))
val_idx = int(.8*len(ico_dict_keys))
train_keys = ico_dict_keys[:tr_idx]
val_keys = ico_dict_keys[tr_idx+1: val_idx]
test_keys = ico_dict_keys[val_idx:]
    
    

with open('Thesis_PreProcessing/dense_reps.pickle', 'rb') as dense_reps:
    dense_dict = pickle.load(dense_reps)
    
X_train = dense_representation_add(ico_dict, dense_dict, train_keys)
Y_train = dict_to_paddedY(ico_dict, train_keys)
X_val = dense_representation_add(ico_dict, dense_dict, val_keys)
Y_val = dict_to_paddedY(ico_dict, val_keys)
X_test = dense_representation_add(ico_dict, dense_dict, test_keys)
Y_test = dict_to_paddedY(ico_dict, test_keys)


g_1 = RNN_Dense_Representation_graph()
train_baseline(g_1, X_train, Y_train, epoch_nums=3, save="/tmp/dense_7.ckpt")
g_2 = RNN_Dense_Representation_graph(batch_size=1, test=True)
losses, loss, recall_20, recall_25, recall_30 = test_baseline(g_2,"/tmp/dense_7.ckpt", X_test, Y_test)

tn.close(current_connection[0], current_connection[1])